In [4]:
import pandas as pd
import datetime
import os
import glob
from scipy.integrate import cumtrapz
from scipy.signal import find_peaks
import numpy as np
from datetime import timedelta

In [37]:
# reference capacity
refc = pd.read_csv('data/testbench_results/testbench_results_first3months.csv')
refc['datetime'] = pd.to_datetime(refc['timestamp'], unit='s')
refc['Q_discharge'] = -refc['Q_discharge']

In [72]:
# battery data
def read_all_csvs(path='data/battery_data'):
    '''Read all CSVs.'''
    csvs = sorted(glob.glob(os.path.join(path, "*.csv")))
    data = pd.concat((pd.read_csv(f) for f in csvs))
    data['datetime'] = pd.to_datetime(data['timestamp'], unit='s')
    return data

data = read_all_csvs()
data = data[(data['timestamp']>datetime.datetime(2020, 4, 1, 9, 49, 49).timestamp()) & (data['timestamp']<datetime.datetime(2020, 9, 30, 0, 0, 0).timestamp())]
data.reset_index(inplace=True)

In [73]:
# integrate current and find peaks/troughs
integral = cumtrapz(y=data['current'], x=data['timestamp'], initial=0).astype(float)/3600
data = data.assign(integral=integral)
peaks = find_peaks(x=integral, width=200)[0] # tune if needed
troughs = find_peaks(x=-integral, width=200)[0] # tune if needed
data['peaks'] = np.nan
data['troughs'] = np.nan
data['peaks'].iloc[peaks] = data['integral'].iloc[peaks]
data['troughs'].iloc[troughs] = data['integral'].iloc[troughs]

/home/iross/.conda/envs/challenge-1/lib/python3.9/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [74]:
# define discharge cycles
d_starts = list(data['peaks'][data['peaks'].notnull()].index[:-1])
d_ends = list()
for start in d_starts:
    end = data['troughs'][(data['troughs'].notnull()) & (data['troughs'].notnull().index>start)].index.min()
    d_ends.append(end)

In [75]:
# define charge cycles
c_starts = list(data['troughs'][data['troughs'].notnull()].index[:-1])
c_ends = list()
for start in c_starts:
    end = data['peaks'][(data['peaks'].notnull()) & (data['peaks'].notnull().index>start)].index.min()
    c_ends.append(end)

In [76]:
# join all into a dataframe
starts = d_starts + c_starts
ends = d_ends + c_ends
cycles = pd.DataFrame(starts, columns=['starts'])
cycles['ends'] = ends

In [77]:
# define cycle features which contribute to damage
# (only voltage used here as example but current and temperature could be used too)
cycles['v_start'] = list(data['voltage'].iloc[cycles['starts'][i]] for i in cycles.index)
cycles['v_end'] = list(data['voltage'].iloc[cycles['ends'][i]] for i in cycles.index)
cycles['v_change'] = list(cycles['v_start'][i]-cycles['v_end'][i] for i in cycles.index)
cycles['timedelta'] = list(data['datetime'].iloc[cycles['ends'][i]]-data['datetime'].iloc[cycles['starts'][i]] for i in cycles.index)
cycles['duration'] = list(timedelta.total_seconds(cycles['timedelta'][i]) for i in cycles.index)
cycles['start_date'] = list(data['datetime'].iloc[cycles['starts'][i]] for i in cycles.index)
cycles.sort_values('starts', ignore_index=True, inplace=True)

In [78]:
# discard test cycles assuming they do no damage
# (or they could also be accounted for)
for i in cycles.index:
    df = pd.DataFrame(data['datetime'].iloc[cycles['starts'][i]:cycles['ends'][i]], columns=['datetime'])
    dt = df['datetime'].diff()[1:]
    if dt.max() > timedelta(seconds=300):
        cycles.drop(index=i, inplace=True)

In [79]:
cycles.sort_values('starts', ignore_index=True, inplace=True)

In [80]:
# add reference capacity
matches = list(cycles.set_index('start_date').index.get_loc(refc['datetime'].iloc[i], method='nearest') for i in refc.index)

In [81]:
cycles['capacity'] = ''
for iref, icyc in enumerate(matches):
    cycles['capacity'].iloc[icyc] = refc['Q_discharge'].iloc[iref]

/home/iross/.conda/envs/challenge-1/lib/python3.9/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [82]:
# define a damage function based on the cycle features
# calculate over time and subtract from the reference capacity
# (below is just a non-sensical example)
cycles['magic'] = ''
cycles['magic'][0] = cycles['capacity'][0]
for i in cycles.index[1:]:
    a = cycles['duration'][i]
    b = abs(cycles['v_change'][i])
    damage = a * b * 0.0000002 * ( 1 - 5*(cycles['capacity'][0] - cycles['magic'][i-1]))
    remain = cycles['magic'][i-1] - damage
    cycles['magic'][i] = remain

# plot stuff
import plotly.graph_objects as go
resampled_data = data.set_index('datetime').resample('60T').mean().reset_index()
fig = go.Figure()
fig.add_trace(go.Scatter(x=resampled_data['datetime'], y=resampled_data['current'], name="current"))
fig.add_trace(go.Scatter(x=resampled_data['datetime'], y=resampled_data['voltage'], name="voltage", yaxis="y2"))
fig.add_trace(go.Scatter(x=refc['datetime'], y=refc['Q_discharge'], name="reference capacity", yaxis="y3"))
fig.add_trace(go.Scatter(x=cycles['start_date'], y=cycles['magic'], name="model capacity", yaxis="y3"))

fig.update_layout(
    xaxis=dict(domain=[0.06, 0.95]), yaxis=dict(title="current", titlefont=dict( color="#1f77b4"), tickfont=dict(color="#1f77b4")),
    yaxis2=dict(title="voltage", titlefont=dict(color="#ff7f0e"), tickfont=dict(color="#ff7f0e"), anchor="free", overlaying="y", side="left", position=0),
    yaxis3=dict(title="capacity", titlefont=dict(color="#d62728"), tickfont=dict(color="#d62728"), anchor="x", overlaying="y", side="right"),
)

fig.update_layout(title_text="Current, voltage, capacity", width=1700)
fig.show()

<ipython-input-82-30dbaeeef16e>:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-82-30dbaeeef16e>:11: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [105]:
predict_dates = pd.Series(['2020-07-11', '2020-08-04', '2020-08-28', '2020-09-22', '2020-10-01'])
predict = pd.DataFrame(index=predict_dates, columns=['capacity'])
for pred in predict.index:
    icyc = cycles.set_index('start_date').index.get_loc(pred, method='nearest')
    predict['capacity'][pred] = np.mean(cycles['magic'][icyc])

In [106]:
predict

,capacity
2020-07-11,11.261304
2020-08-04,11.24564
2020-08-28,11.234289
2020-09-22,11.22603
2020-10-01,11.223995
